## Modules

In [ ]:
# import numpy as np
import pandas as pd; pd.set_option("display.max_colwidth", 1000)

from GridPythonModule import *
from GPM_wrapper import *

#from tqdm.notebook import tqdm
# ----- Debugger -----
# from IPython.core.debugger import Pdb; Pdb().set_trace()

## Scratch

In [ ]:
mg = Grid(*[[3, 2, 5, 4, 7, 6, 0, 1], [0, 4, 3, 6, 5, 1, 2, 7]]); mg.draw()
print(reduce(mg.get_PD()))

In [ ]:
nontrivknot = None
while nontrivknot is None:
    G = generate_random_grid(20)
    sG = set(sum(simplify_grid(G), []))
    if number_of_components(G) == 1 and sG != {0,1}:
        nontrivknot = G
#---
mg = Grid(*G); mg.draw()
PD = mg.get_PD()
print(reduce(PD))
#---
mgs = Grid(*simplify_grid(G)); mgs.draw()
PDs = mgs.get_PD()
print(reduce(PDs))

# Validation on outputs of pd_to_hfk and a code of myself converting grids to PD notations

## Preparing DataFrames for input and output

### Input DataFrame `knotinfo_df` (from KnotInfo)

In [ ]:
knotinfo_df = pd.read_csv("./data/knotinfo_3-8.csv").set_index('Name')
display(knotinfo_df)

### Output DataFrame `hfk_df` (generated by pd_to_hfk from PD notations given from grid diagrams)

In [ ]:
from knot_floer_homology import *

list_of_knots = ['3_1', '4_1', '5_1', '5_2', '6_1', '6_2', '6_3', '7_1', '7_2', '7_3', '7_4', '7_5', '7_6', '7_7', '8_1', '8_2', '8_3', '8_4', '8_5', '8_6', '8_7', '8_8', '8_9', '8_10', '8_11', '8_12', '8_13', '8_14', '8_15', '8_16', '8_17', '8_18', '8_19', '8_20', '8_21']

def replace_to_hfk(knot_name):
    G = load_knot(knot_name); #draw_grid(G)
    PD = Grid(*G).get_PD(); #print(PD)
    hfk = pd_to_hfk(PD); #print(hfk)
    hfk['Name'] = knot_name
    hfk['PD Notation'] = PD
    hfk['Grid'] = G
    return hfk

hfk_list = list(map(replace_to_hfk, list_of_knots))
hfk_df = pd.DataFrame(hfk_list).set_index('Name'); display(hfk_df)

## Validation

In [ ]:
for name, ranks in hfk_df['ranks'].items():
    print(f"{name}: {[list(ranks.keys())[i] for i in (0,-1)]}")

In [ ]:
comparing_ranks = pd.concat([hfk_df[['fibered','seifert_genus', 'ranks']], knotinfo_df[['HFK Polyomial', 'Genus-3D']]], axis=1); display(comparing_ranks.query("fibered"))

# print(comparing_ranks.loc['8_5'])
# print(comparing_ranks.loc['8_16'])

In [ ]:
import ast

print(hfk_df.loc['8_5','PD Notation'])
ki_pd = knotinfo_df.loc['8_5','PD Notation'].replace(';',','); #print(ki_pd)
ki_pd = ast.literal_eval(ki_pd)
ki_pd = [tuple(map(lambda x: x-1, l)) for l in ki_pd]
print(ki_pd)

In [ ]:
cols_list = [('Epsilon', 'epsilon'), ('Nu', 'nu'), ('PD Notation', 'PD Notation')]
for name in list_of_knots:
    for i, df in enumerate((knotinfo_df, hfk_df)):
        print(df.loc[name, cols[i]])